In [1]:
import sys
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import svm, metrics, preprocessing
from sklearn.model_selection import train_test_split, cross_val_score

from scipy.fftpack import fft
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss

from sklearn.svm import SVC
from sklearn import decomposition

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
import random

In [2]:
np.random.seed(0)

In [3]:
y = np.zeros(180)
y[0:45] = 1
y[45:90] = 2
y[90:135] = 3
y[135:180] = 4

In [4]:
N = 4*2500
sampling_frequency = 2500 # Hz
channelAccuracies = {}
allData = np.zeros((180, 40, 68))

for channelNumber in range(1, 41):
  x = pd.read_csv("BCI_Project/Snigdha_time_imagine/time_channel_{}.csv".format(channelNumber), header=None)
  x = np.array(x)
  x = x[1:, 2500:12500:10]
  
  x_fft = fft(x)
  x_fft = np.abs(x_fft[:int(N/20)])
  x_fft = x_fft[:, 32:100]
  
  allData[:, channelNumber-1, :] = x_fft

In [5]:
x_train, x_test, y_train, y_test = train_test_split(allData, y, test_size = 0.3, random_state = 0)

In [6]:
scaledx_train = np.zeros(np.shape(x_train))
scaledx_test = np.zeros(np.shape(x_test))

for channelNumber in range(1, 41):  
  scaler = preprocessing.StandardScaler().fit(x_train[:, channelNumber-1, :])
  scaledx_train[:, channelNumber-1, :] = scaler.transform(x_train[:, channelNumber-1, :])
  scaledx_test[:, channelNumber-1, :] = scaler.transform(x_test[:, channelNumber-1, :])

In [7]:
sorted_accuracies = sorted(channelAccuracies.items(), key=lambda kv: kv[1])
print(sorted_accuracies)

[]


In [107]:
selectedChannelsData_train = scaledx_train[:, [19, 37, 20, 12, 38, 4, 36], :]
selectedChannelsData_test = scaledx_test[:, [19, 37, 20, 12, 38, 4, 36], :]
print(np.shape(selectedChannelsData_train))
selectedChannelsData_train = np.reshape(selectedChannelsData_train, (126, -1))
print(np.shape(selectedChannelsData_train))
print("-------")
print(np.shape(selectedChannelsData_test))
selectedChannelsData_test = np.reshape(selectedChannelsData_test, (54, -1))
print(np.shape(selectedChannelsData_test))

(126, 7, 68)
(126, 476)
-------
(54, 7, 68)
(54, 476)


In [8]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [2**(-11), 2**(-9), 2**(-5), 2**(-3), 2**(-1), 2**1, 2**3],
                   'C': [2**(-5), 2**(-1), 2**3, 2**5, 2**7, 2**15]}]

clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=10,
                     scoring='accuracy')
clf.fit(selectedChannelsData_train, y_train)

NameError: name 'selectedChannelsData_train' is not defined

In [109]:
clf = svm.SVC(C = clf.best_params_['C'], kernel = clf.best_params_['kernel'], gamma = clf.best_params_['gamma'])
scores = cross_val_score(clf, selectedChannelsData_train, y_train, cv=10)
print("Train Accuracy: ", scores.mean())

Train Accuracy:  0.6078671328671329


In [110]:
clf.fit(selectedChannelsData_train, y_train)
test_predictions = clf.predict(selectedChannelsData_test)
accuracy = np.mean(test_predictions == y_test)

In [111]:
from sklearn.metrics import confusion_matrix
results = confusion_matrix(y_test, test_predictions) 
print(results)

[[ 7  6  0  0]
 [ 5  7  0  0]
 [ 0  2  9  2]
 [ 1  0  5 10]]


In [112]:
print("Test accuracy: ", accuracy)

Test accuracy:  0.6111111111111112


In [141]:
run_input = selectedChannelsData_test[5, :]
run_input = run_input.reshape((1, -1))
print(np.shape(run_input))
run_pred = clf.predict(run_input)
run_label = y_test[5]

if(run_label == 1):
    print("Car should go forward")
elif(run_label == 2):
    print("Car should go backwards")
elif(run_label == 3):
    print("Car should go right")
elif(run_label == 4):
    print("Car should go left")

(1, 476)
Car should go backwards


In [153]:
import serial
import time
time.sleep(2)
ArduinoSerial = serial.Serial('/dev/cu.usbmodem14101',9600)

print(run_pred)

if(run_pred == 1):
    ArduinoSerial.write('1'.encode())
    ArduinoSerial.write('0'.encode())
    
if(run_pred == 2):
    ArduinoSerial.write('2'.encode())
    ArduinoSerial.write('0'.encode())
    
if(run_pred == 3):
    ArduinoSerial.write('3'.encode())
    ArduinoSerial.write('0'.encode())
    
if(run_pred == 4):
    ArduinoSerial.write('4'.encode())
    ArduinoSerial.write('0'.encode())

4


In [143]:
print(y_test)

[3. 1. 3. 4. 4. 2. 2. 4. 4. 3. 1. 4. 4. 1. 1. 3. 4. 4. 1. 3. 4. 2. 2. 1.
 3. 4. 3. 4. 4. 2. 1. 1. 3. 3. 2. 1. 2. 2. 2. 4. 3. 1. 1. 3. 2. 2. 1. 4.
 3. 4. 1. 2. 4. 3.]
